In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',200)
pd.set_option('display.width',1000)

## 1. Carregamento e Pré-processamento dos Dados

In [4]:
# Carregando a base de dados
ts_raw = pd.read_csv('https://raw.githubusercontent.com/abnr/ml-data/main/stocks_ABBV.csv', index_col=[0])

# Selecionando e renomeando as colunas para o padrão Prophet
ts = ts_raw[['Date', 'Close']].copy()
ts.rename(columns = {'Date':'ds', 'Close':'y'}, inplace = True)

# Convertendo 'ds' para datetime e ordenando
ts['ds'] = pd.to_datetime(ts['ds'])
ts = ts.sort_values('ds')

display(ts.head())
display(ts.tail())

,ds,y
1676,2013-01-02,35.119999
1743,2013-01-03,34.830002
1792,2013-01-04,34.389999
1759,2013-01-07,34.459999
1757,2013-01-08,33.709999


,ds,y
1730,2020-03-26,73.830002
1613,2020-03-27,72.669998
1508,2020-03-30,75.239998
1785,2020-03-31,76.190002
1648,2020-04-01,73.419998


## 2. Definição dos Modelos (m0 a m7)

### Modelos Originais (m0 a m4)

In [5]:
# m0: Modelo Padrão
m0 = Prophet()

# m1: changepoint_prior_scale = 0.5 (Maior flexibilidade na tendência)
m1 = Prophet(changepoint_prior_scale=0.5)

# m2: changepoint_prior_scale = 0.99 (Máxima flexibilidade na tendência)
m2 = Prophet(changepoint_prior_scale=0.99)

# m3: ch# m3: Mais changepoints em uma faixa maior (n_changepoints=50, changepoint_range=0.9)
m3 = Prophet(n_changepoints=50, changepoint_range=0.9)

# m4: ch# m4: changepoint_prior_scale intermediário (0.08)
m4 = Prophet(changepoint_prior_scale=0.08)

### Novos Modelos (m5 a m7)

In [6]:
m5 = Prophet(yearly_seasonality=False, weekly_seasonality=True, seasonality_mode='multiplicative', seasonality_prior_scale=10.0)
m5.add_seasonality(name='yearly', period=365.25, fourier_order=20)

m6 = Prophet(yearly_seasonality=False, weekly_seasonality=False, changepoint_prior_scale=0.005)

m7 = Prophet(changepoint_prior_scale=0.1)
m7.add_seasonality(name='quarterly', period=91.25, fourier_order=5)

## 3. Treino/Teste e Coleta de Métricas

**ATENÇÃO:** Professor, não conseguimos rodar a cross validation, o notebook não estava conseguindo executar o código, tomava timed out.


In [7]:
# Lista de modelos
models = {
    'm0': m0, 'm1': m1, 'm2': m2, 'm3': m3, 'm4': m4,
    'm5': m5, 'm6': m6, 'm7': m7
}

train_size = int(len(ts) * 0.8)
train_df = ts[:train_size]
test_df = ts[train_size:]

# Dataframe de Validação (apenas MSE, sem STD)
df_validacao = pd.DataFrame(columns = (['modelo', 'MSE']))

for name, model in models.items():
    print(f"Iniciando treino para o modelo {name}...")

    # Treino
    model.fit(train_df)

    # Previsão no conjunto de teste
    forecast = model.predict(test_df)

    # Cálculo do MSE no conjunto de teste
    try:
        # Garantir que os dataframes estão alinhados
        y_true = test_df['y'].reset_index(drop=True)
        y_pred = forecast['yhat'].reset_index(drop=True)

        min_len = min(len(y_true), len(y_pred))
        y_true = y_true[:min_len]
        y_pred = y_pred[:min_len]

        mse = mean_squared_error(y_true, y_pred)

        # Armazenar para a tabela de validação
        new_row = pd.DataFrame([{'modelo': name,
                                 'MSE': np.round(mse, 2)}])

        df_validacao = pd.concat([df_validacao, new_row], ignore_index=True)
        print(f"Modelo {name} concluído. MSE: {mse:.2f}")

    except Exception as e:
        print(f"Erro no cálculo do MSE para o modelo {name}: {e}")
        new_row = pd.DataFrame([{'modelo': name, 'MSE': 'Erro'}])
        df_validacao = pd.concat([df_validacao, new_row], ignore_index=True)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Iniciando treino para o modelo m0...


/tmp/ipython-input-3559657954.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_validacao = pd.concat([df_validacao, new_row], ignore_index=True)
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Modelo m0 concluído. MSE: 1460.16
Iniciando treino para o modelo m1...


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Modelo m4 concluído. MSE: 1373.36
Iniciando treino para o modelo m5...


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Modelo m5 concluído. MSE: 1731.74
Iniciando treino para o modelo m6...


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Modelo m6 concluído. MSE: 1850.84
Iniciando treino para o modelo m7...
Modelo m7 concluído. MSE: 1438.86

Treino e avaliação de todos os modelos concluída.


## 4. Tabela de Comparação dos Modelos (m0 a m7)

A tabela `df_validacao` compara o MSE (Erro Quadrático Médio) no conjunto de teste (20% final da série) para todos os modelos.

In [8]:
# Exibindo a tabela de validação final
df_validacao.sort_values(by='MSE')

,modelo,MSE
3,m3,266.31
4,m4,1373.36
7,m7,1438.86
0,m0,1460.16
1,m1,1467.49
2,m2,1483.02
5,m5,1731.74
6,m6,1850.84


## 5. Indicação do Melhor Modelo e Justificativa

In [10]:
melhor_modelo = df_validacao.sort_values(by='MSE').iloc[0]
print("Melhor Modelo:")
display(melhor_modelo)

Melhor Modelo:


,3
modelo,m3
MSE,266.31


Com base nos resultados do treino/teste simples (80/20), o modelo com o menor **MSE** é o melhor para este corte de dados.

### Justificativa para a Escolha do Melhor Modelo

O **MSE (Erro Quadrático Médio)** é a métrica utilizada para avaliar a precisão do modelo no conjunto de teste.

| Modelo | MSE | Hiperparâmetros Chave |
| :--- | :---: | :--- |
| m3 | [266.31] | `changepoints=['2019-08-01']` |
| m4 | [1373.36] | `changepoints=['2018-01-01']` |
| m7 | [1438.86] | `changepoint_prior_scale=0.1`, `add_seasonality(quarterly)` |
| **m0** | [1460.16] | Padrão (default) |
| m1 | [1467.49] | `changepoint_prior_scale=0.5` |
| m2 | [1483.02] | `changepoint_prior_scale=0.99` |
| m5 | [1731.74] | `seasonality_mode='multiplicative'`, `yearly_fourier_order=20` |
| m6 | [1850.84] | `changepoint_prior_scale=0.005`, Sazonalidades Desativadas |



**Conclusão:**

O **Melhor Modelo** foi o m3, baseado no MS3. Como a validação cruzada foi removida, este resultado representa a performance do modelo apenas no último corte de 20% dos dados. Em um ambiente de produção, a validação cruzada seria essencial pra garantir a robustez e uma análise mais precisa.